# Insurance cost prediction using linear regression

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from: https://www.kaggle.com/mirichoi0218/insurance


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lectures. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ml/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ml/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ml/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ml/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ml/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [3]:
# Uncomment and run the commands below if imports fail
# !conda install numpy pytorch torchvision cpuonly -c pytorch -y
# !pip install matplotlib --upgrade --quiet
!pip install jovian --upgrade --quiet

     |████████████████████████████████| 92kB 2.5MB/s 


In [0]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [0]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [6]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [7]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [0]:
your_name = "pranav" # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [0]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [11]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()
dataframe.shape

(1271, 7)

Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [12]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [15]:
num_cols = len(dataframe.columns)
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [0]:
input_cols = ['age',	'sex',	'bmi',	'children',	'smoker',	'region',	'charges']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [0]:
categorical_cols = ['sex', 'smoker', 'region']

**Q: What are the column titles of output/target variable(s)?**

In [0]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [20]:
#n=dataframe['charges',:].mean()
n = dataframe['charges'].mean()
print(n)

12925.252525036676


Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [21]:
jovian.commit(project=project_name, environment=None)

[jovian] Error: Failed to detect Jupyter notebook or Python script. Skipping..


## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [0]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [23]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[5.60000000e+01, 0.00000000e+00, 4.53948000e+01, ...,
         0.00000000e+00, 2.00000000e+00, 1.07579963e+04],
        [4.20000000e+01, 0.00000000e+00, 2.84829000e+01, ...,
         0.00000000e+00, 1.00000000e+00, 7.77654932e+03],
        [1.80000000e+01, 1.00000000e+00, 3.82299000e+01, ...,
         1.00000000e+00, 0.00000000e+00, 3.35793054e+04],
        ...,
        [2.80000000e+01, 1.00000000e+00, 4.03959000e+01, ...,
         0.00000000e+00, 0.00000000e+00, 3.17078125e+03],
        [3.40000000e+01, 0.00000000e+00, 2.68584000e+01, ...,
         0.00000000e+00, 0.00000000e+00, 4.84260511e+03],
        [4.60000000e+01, 1.00000000e+00, 2.94120000e+01, ...,
         0.00000000e+00, 3.00000000e+00, 9.79406090e+03]]),
 array([[10757.996266 ],
        [ 7776.5493155],
        [33579.3054305],
        ...,
        [ 3170.7812505],
        [ 4842.605108 ],
        [ 9794.0609   ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [0]:
inputs = torch.from_numpy(inputs_array)
targets = torch.from_numpy(targets_array)

In [25]:
inputs.dtype, targets.dtype

(torch.float64, torch.float64)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [26]:
dataset = TensorDataset(inputs, targets)
len(dataset)

1271

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets. **

In [0]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset,[1000,271]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [0]:
batch_size = 64

In [0]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [30]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[1.8000e+01, 0.0000e+00, 3.5739e+01, 4.0000e+00, 0.0000e+00, 0.0000e+00,
         4.4244e+03],
        [4.4000e+01, 0.0000e+00, 4.3388e+01, 0.0000e+00, 1.0000e+00, 2.0000e+00,
         4.7419e+04],
        [3.3000e+01, 0.0000e+00, 4.8952e+01, 3.0000e+00, 0.0000e+00, 1.0000e+00,
         6.1702e+03],
        [3.6000e+01, 0.0000e+00, 2.5764e+01, 2.0000e+00, 1.0000e+00, 3.0000e+00,
         1.8050e+04],
        [2.8000e+01, 1.0000e+00, 3.0757e+01, 2.0000e+00, 0.0000e+00, 0.0000e+00,
         4.3020e+03],
        [2.0000e+01, 0.0000e+00, 3.6389e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         2.1937e+03],
        [6.3000e+01, 1.0000e+00, 3.8372e+01, 3.0000e+00, 0.0000e+00, 2.0000e+00,
         1.4707e+04],
        [5.9000e+01, 0.0000e+00, 3.0216e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.2431e+04],
        [2.9000e+01, 0.0000e+00, 4.4266e+01, 3.0000e+00, 0.0000e+00, 2.0000e+00,
         4.9841e+03],
        [2.2000e+01, 1.0000e+00, 4.2887e+01, 1.0000e+00, 1.0000e+

Let's save our work by committing to Jovian.

In [0]:
jovian.commit(project=project_name, environment=None)

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [0]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [0]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                 # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.smooth_l1_loss(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.smooth_l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [0]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [49]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1379,  0.2514, -0.0972,  0.1083, -0.2971,  0.1502,  0.0079]],
        requires_grad=True), Parameter containing:
 tensor([0.2796], requires_grad=True)]

One final commit before we train the model.

In [40]:
jovian.commit(project=project_name, environment=None)

[jovian] Error: Failed to detect Jupyter notebook or Python script. Skipping..


## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [0]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [51]:
result = evaluate(model.double(), val_loader) # Use the the evaluate function
print(result)

{'val_loss': 11892.462649784424}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [52]:
epochs = 1000
lr = 1e-2
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 456534.3789
Epoch [40], val_loss: 1254485.9212
Epoch [60], val_loss: 317844.2959
Epoch [80], val_loss: 1689804.3278
Epoch [100], val_loss: 539665.6329
Epoch [120], val_loss: 190793.8421
Epoch [140], val_loss: 1666278.3974
Epoch [160], val_loss: 1254943.8833
Epoch [180], val_loss: 1245173.5884
Epoch [200], val_loss: 1412675.9032
Epoch [220], val_loss: 875446.8504
Epoch [240], val_loss: 1347675.9945
Epoch [260], val_loss: 1016212.9195
Epoch [280], val_loss: 415076.2581
Epoch [300], val_loss: 900875.6431
Epoch [320], val_loss: 1828841.1054
Epoch [340], val_loss: 1190401.5570
Epoch [360], val_loss: 1171179.6722
Epoch [380], val_loss: 1442081.0026
Epoch [400], val_loss: 150687.0626
Epoch [420], val_loss: 190133.0195
Epoch [440], val_loss: 810667.7050
Epoch [460], val_loss: 878261.5472
Epoch [480], val_loss: 1016575.9596
Epoch [500], val_loss: 844541.6664
Epoch [520], val_loss: 847696.7590
Epoch [540], val_loss: 1052961.2882
Epoch [560], val_loss: 144063.8977
Epoch [580

In [53]:
epochs = 500
lr = 1e-3
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 82125.0960
Epoch [40], val_loss: 129697.8377
Epoch [60], val_loss: 59798.2615
Epoch [80], val_loss: 139762.5109
Epoch [100], val_loss: 4348.4623
Epoch [120], val_loss: 73929.6834
Epoch [140], val_loss: 99200.1220
Epoch [160], val_loss: 10128.5313
Epoch [180], val_loss: 154567.9487
Epoch [200], val_loss: 118240.7046
Epoch [220], val_loss: 82875.4672
Epoch [240], val_loss: 3186.4293
Epoch [260], val_loss: 39655.9604
Epoch [280], val_loss: 128981.6608
Epoch [300], val_loss: 28608.8780
Epoch [320], val_loss: 155594.7820
Epoch [340], val_loss: 69657.6742
Epoch [360], val_loss: 119264.5985
Epoch [380], val_loss: 2734.8231
Epoch [400], val_loss: 130159.7582
Epoch [420], val_loss: 167059.6296
Epoch [440], val_loss: 14742.2714
Epoch [460], val_loss: 33394.1624
Epoch [480], val_loss: 98203.9470
Epoch [500], val_loss: 119379.3952


In [54]:
epochs = 500
lr = 1e-5
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 858.3311
Epoch [40], val_loss: 1254.8069
Epoch [60], val_loss: 629.2555
Epoch [80], val_loss: 1334.2829
Epoch [100], val_loss: 103.4213
Epoch [120], val_loss: 385.6753
Epoch [140], val_loss: 535.4225
Epoch [160], val_loss: 1015.9167
Epoch [180], val_loss: 1373.1121
Epoch [200], val_loss: 546.8949
Epoch [220], val_loss: 1134.0033
Epoch [240], val_loss: 578.4479
Epoch [260], val_loss: 555.4667
Epoch [280], val_loss: 376.3992
Epoch [300], val_loss: 450.6930
Epoch [320], val_loss: 646.0845
Epoch [340], val_loss: 1027.5553
Epoch [360], val_loss: 844.8696
Epoch [380], val_loss: 1284.0237
Epoch [400], val_loss: 401.6479
Epoch [420], val_loss: 740.6762
Epoch [440], val_loss: 1366.8649
Epoch [460], val_loss: 657.2376
Epoch [480], val_loss: 1460.0624
Epoch [500], val_loss: 440.2810


In [55]:
epochs = 300
lr = 1e-6
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 55.0594
Epoch [40], val_loss: 54.6516
Epoch [60], val_loss: 101.9657
Epoch [80], val_loss: 86.8026
Epoch [100], val_loss: 53.4193
Epoch [120], val_loss: 53.3811
Epoch [140], val_loss: 58.4409
Epoch [160], val_loss: 53.2098
Epoch [180], val_loss: 52.4928
Epoch [200], val_loss: 52.5689
Epoch [220], val_loss: 54.8271
Epoch [240], val_loss: 51.8773
Epoch [260], val_loss: 52.3193
Epoch [280], val_loss: 74.9306
Epoch [300], val_loss: 53.0463


In [56]:
epochs = 200
lr = 1e-7
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 52.1146
Epoch [40], val_loss: 54.0220
Epoch [60], val_loss: 52.0299
Epoch [80], val_loss: 51.6565
Epoch [100], val_loss: 52.5211
Epoch [120], val_loss: 52.6080
Epoch [140], val_loss: 52.1379
Epoch [160], val_loss: 52.1629
Epoch [180], val_loss: 51.6922
Epoch [200], val_loss: 52.6308


**Q: What is the final validation loss of your model?**

In [59]:
val_loss = history5[-1]
print(val_loss)

{'val_loss': 52.630813234842364}


Let's log the final validation loss to Jovian and commit the notebook

In [60]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Please enter your API key ( from https://jovian.ml/ ):
API KEY: ··········
[jovian] Metrics logged.


In [61]:
jovian.commit(project=project_name, environment=None)

[jovian] Error: Failed to detect Jupyter notebook or Python script. Skipping..


Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [0]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [63]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([5.4000e+01, 0.0000e+00, 4.0829e+01, 3.0000e+00, 0.0000e+00, 1.0000e+00,
        1.2120e+04], dtype=torch.float64)
Target: tensor([12120.4321], dtype=torch.float64)
Prediction: tensor([12046.5041], dtype=torch.float64)


In [64]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([1.8000e+01, 0.0000e+00, 3.8623e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00,
        1.1138e+04], dtype=torch.float64)
Target: tensor([11138.1558], dtype=torch.float64)
Prediction: tensor([11178.5444], dtype=torch.float64)


In [65]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([2.5000e+01, 1.0000e+00, 2.9891e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        2.6397e+03], dtype=torch.float64)
Target: tensor([2639.6812], dtype=torch.float64)
Prediction: tensor([2610.2434], dtype=torch.float64)


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to clean up & replicate this notebook (or [this one](https://jovian.ml/aakashns/housing-linear-minimal), or [this one](https://jovian.ml/aakashns/mnist-logistic-minimal) ) for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patters in machine learning from problem-specific details.

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ml/forum/t/share-your-work-here-assignment-2/4931

In [0]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Please enter your API key ( from https://jovian.ml/ ):
API KEY: ········
[jovian] Creating a new project "aakashns/02-insurance-linear-regression"
[jovian] Uploading notebook..
[jovian] Committed successfully! https://jovian.ml/aakashns/02-insurance-linear-regression


<IPython.core.display.Javascript object>